# Run stargeo queries for hetio-ind diseases

In [1]:
import logging
import os
import gzip

import pandas
import easydict

import starapi.analysis
import starapi.conf

In [2]:
starapi.analysis.logger.propagate = False
starapi.conf.configure('data')

In [3]:
query_df = pandas.read_table('data/queries.tsv')
query_df.head()

,slim_id,slim_name,case_query,control_query
0,DOID:0050156,idiopathic pulmonary fibrosis,IPF == 'IPF',IPF_control == 'IPF_control'
1,DOID:0050741,alcohol dependence,alcoholism == 'alcoholism',alcoholism_control == 'alcoholism_control'
2,DOID:0050742,nicotine dependence,Smoker == 'Smoker',Smoker_control == 'Smoker_control'
3,DOID:1024,leprosy,borderline_leprosy == 'borderline_leprosy' or ...,leprosy_control == 'leprosy_control'
4,DOID:10283,prostate cancer,PC == 'PC',PC_control == 'PC_control' or PC_tissue_contro...


In [4]:
errors = list()

for i, row in query_df.iterrows():
    print i, row.slim_id, row.slim_name

    name = row.slim_id.replace(':', '_')
    params = easydict.EasyDict(
        analysis_name = name,
        case_query = row.case_query,
        control_query = row.control_query,
        modifier_query = '',
        min_samples = 3,
    )
    
    directory = os.path.join('data', 'doslim', params.analysis_name)
    if not os.path.isdir(directory):
        os.mkdir(directory)

    logfile = logging.FileHandler(os.path.join(directory, 'log.txt'))
    logfile.setLevel(logging.DEBUG)
    starapi.analysis.logger.addHandler(logfile)
    try:
        sample_df, fold_df, balanced_perm_df, perm_df = starapi.analysis.perform_analysis(params)
        for df in fold_df, balanced_perm_df:
            if df is not None:
                df.reset_index(inplace=True)
        
    except Exception as e:
        sample_df, fold_df, balanced_perm_df, perm_df = None, None, None, None
        errors.append({'slim_id': row.slim_id, 'name': name, 'error': e})
        print(e)
    starapi.analysis.logger.removeHandler(logfile)
    
    # write files
    files = [
        (sample_df, 'samples.tsv', open),
        (fold_df, 'fold_change.tsv.gz', gzip.open),
        (balanced_perm_df, 'balanced_permutation.tsv.gz', gzip.open),
    ]
    for df, filename, opener in files:
        if df is None:
            continue
        path = os.path.join(directory, filename)
        with opener(path, 'wt') as write_file:
            df.to_csv(write_file, index=False, sep='\t', float_format='%.5g')


Started DOID_0050156 analysis
/home/dhimmels/anaconda3/envs/stargeo/lib/python2.7/site-packages/pandas/core/indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
915.86 ms in get_analysis_df("IPF == 'IPF'", "IPF_control == 'IPF_c..., '')
920.37 ms in Loading dataframe for DOID_0050156
Matching sources: 12
Stats: 12 sources, 12 series, 8 platforms, 679 samples
Loading data and calculating fold change for DOID_0050156
Loading data for GSE10667, id = 179
733.09 ms in get_probes(10)
 18.13 s in load_gse( series_id platform_id..., 179, False)
/home/dhimmels/Documents/serg/rephetio/construct/star_api/starapi/analysis.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

0 DOID:0050156 idiopathic pulmonary fibrosis
1

Started DOID_0050741 analysis
935.05 ms in get_analysis_df("alcoholism == 'alcoho..., "alcoholism_control ==..., '')
939.27 ms in Loading dataframe for DOID_0050741
Matching sources: 8
Stats: 8 sources, 8 series, 8 platforms, 435 samples
Loading data and calculating fold change for DOID_0050741
Loading data for GSE10356, id = 95
 39.47 ms in get_probes(36)
 22.10 s in load_gse( series_id platform_id..., 95, False)
205.21 ms in get_gene_fold_change(<Gse GSE10356>, False, 0, None)
Loading data for GSE20568, id = 2733
 42.28 ms in get_probes(689)
  2.34 s in load_gse( series_id platform_id..., 2733, False)
324.24 ms in get_gene_fold_change(<Gse GSE20568>, False, 0, None)
Loading data for GSE29555, id = 5002
492.71 ms in get_probes(389)
 21.88 s in load_gse( series_id platform_id..., 5002, False)
  5.61 s in get_gene_fold_change(<Gse GSE29555>, False, 0, None)
Loading data for GSE44456, id = 8480
411.77 ms in get_probes(283)
 12.21 s in load_gse( series_id platform_id..., 8480, False)
  2.

 DOID:0050741 alcohol dependence
2

Started DOID_0050742 analysis
  1.08 s in get_analysis_df("Smoker == 'Smoker'", "Smoker_control == 'Sm..., '')
  1.08 s in Loading dataframe for DOID_0050742
Matching sources: 22
Excluded 3 sources as single-class
Stats: 19 sources, 17 series, 6 platforms, 1423 samples
Loading data and calculating fold change for DOID_0050742
Loading data for GSE10006, id = 4
631.65 ms in get_probes(4)
 45.85 s in load_gse( series_id platform_id..., 4, False)
  7.15 s in get_gene_fold_change(<Gse GSE10006>, False, 0, None)
Loading data for GSE11784, id = 450
649.63 ms in get_probes(4)
 85.42 s in load_gse( series_id platform_id..., 450, False)
 11.72 s in get_gene_fold_change(<Gse GSE11784>, False, 0, None)
Loading data for GSE11906, id = 487
632.96 ms in get_probes(4)
114.28 s in load_gse( series_id platform_id..., 487, False)
 15.96 s in get_gene_fold_change(<Gse GSE11906>, False, 0, None)
Loading data for GSE19407, id = 2430
667.29 ms in get_probes(4)
 64.76 s in load_gse( series_id platform_id..., 

...

 DOID:9744 type 1 diabetes mellitus
64

Started DOID_986 analysis
812.39 ms in get_analysis_df("alopecia_areata == 'a..., "alopecia_areata_contr..., '')
816.68 ms in Loading dataframe for DOID_986
Matching sources: 2
Stats: 2 sources, 2 series, 1 platforms, 17 samples
Loading data and calculating fold change for DOID_986
Loading data for GSE45512, id = 8684
628.17 ms in get_probes(4)
 10.47 s in load_gse( series_id platform_id..., 8684, False)
  2.43 s in get_gene_fold_change(<Gse GSE45512>, False, 0, None)
Loading data for GSE58573, id = 10697
621.48 ms in get_probes(4)
  8.49 s in load_gse( series_id platform_id..., 10697, False)
  2.23 s in get_gene_fold_change(<Gse GSE58573>, False, 0, None)
 24.13 s in Load/fold for DOID_986
Meta-Analyzing DOID_986
351.41 s in meta analysis of real data for DOID_986
238.83 ms in meta analysis of permutations for DOID_986
108.96 mks in get_balanced_permutations({'analysis_name': 'DOI...,  C H H_lower H_upper I..., Empty DataFrame Column...)
352.32 s in Meta analysis for DOID_986
DONE DOI

 DOID:986 alopecia areata
65

Started DOID_9970 analysis
994.06 ms in get_analysis_df("obesity == 'obesity'", "obesity_control == 'o..., '')
998.34 ms in Loading dataframe for DOID_9970
Matching sources: 43
Excluded 23 sources as single-class
Stats: 20 sources, 14 series, 16 platforms, 561 samples
Loading data and calculating fold change for DOID_9970
Loading data for GSE12050, id = 521
666.94 ms in get_probes(332)
 12.78 s in load_gse( series_id platform_id..., 521, False)
  2.55 s in get_gene_fold_change(<Gse GSE12050>, False, 0, None)
Loading data for GSE15524, id = 1461
343.06 ms in get_probes(500)
  7.89 s in load_gse( series_id platform_id..., 1461, False)
  1.59 s in get_gene_fold_change(<Gse GSE15524>, False, 0, None)
Loading data for GSE2508, id = 3892
288.37 ms in get_probes(7)
290.41 ms in get_probes(8)
256.84 ms in get_probes(13)
220.59 ms in get_probes(14)
198.40 ms in get_probes(15)
246.18 ms in get_probes(16)
 26.13 s in load_gse( series_id platform_id..., 3892, False)
  4.95 s in get_gene_fold_chang

 DOID:9970 obesity


In [13]:
path = 'data/errors.csv'
if error_df.empty:
    if os.path.exists(path):
        os.remove(path)
else:
    error_df = pandas.DataFrame(errors)
    error_df = query_df.merge(error_df)
    error_df.to_csv(path, index=False)

error_df

""
